In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install duckdb --no-index --find-links=/kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg
!pip install numpy --no-index --find-links=/kaggle/input/suman-dicom-tpu-supporting-combo/dicom_tpu_supporting_combo/kaggle/working/mysitepackages/pylibjpeg_libjpeg/numpy-2.3.3-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl

!pip install pylibjpeg-libjpeg==2.2.0 --no-index --find-links=/kaggle/input/suman-dicom-tpu-supporting-combo/dicom_tpu_supporting_combo/kaggle/working/mysitepackages/pylibjpeg_libjpeg
!pip install pylibjpeg-openjpeg==2.3.0 --no-index --find-links=/kaggle/input/suman-dicom-tpu-supporting-combo/dicom_tpu_supporting_combo/kaggle/working/mysitepackages/pylibjpeg_openjpeg

!pip install python-gdcm --no-index --find-links=/kaggle/input/suman-dicom-tpu-supporting-combo/pydicom_working_combo/kaggle/working/mysitepackages/python_gdcm
!pip install pylibjpeg --no-index --find-links=/kaggle/input/suman-dicom-tpu-supporting-combo/pydicom_working_combo/kaggle/working/mysitepackages/pylibjpeg

!pip install polars --no-index --find-links=/kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg
!pip install pydicom --no-index --find-links=/kaggle/input/suman-dicom-tpu-supporting-combo/pydicom_working_combo/kaggle/working/mysitepackages/pydicom

!pip install tensorflow-io==0.37.1 --no-index --find-links=/kaggle/input/suman-tensorflow-working-combo/kaggle/working/mysitepackages/tensorflow_io

!pip install tensorflow==2.16.1 --no-index --find-links=/kaggle/input/suman-tensorflow-working-combo/kaggle/working/mysitepackages/tensorflow

Looking in links: /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg
Looking in links: /kaggle/input/suman-dicom-tpu-supporting-combo/dicom_tpu_supporting_combo/kaggle/working/mysitepackages/pylibjpeg_libjpeg/numpy-2.3.3-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl
Looking in links: /kaggle/input/suman-dicom-tpu-supporting-combo/dicom_tpu_supporting_combo/kaggle/working/mysitepackages/pylibjpeg_libjpeg
Processing /kaggle/input/suman-dicom-tpu-supporting-combo/dicom_tpu_supporting_combo/kaggle/working/mysitepackages/pylibjpeg_libjpeg/pylibjpeg_libjpeg-2.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Looking in links: /kaggle/input/suman-dicom-tpu-supporting-combo/dicom_tpu_supporting_combo/kaggle/working/mysitepackages/pylibjpeg_openjpeg
Processing /kaggle/input/suman-dicom-tpu-supporting-combo/dicom_tpu_supporting_combo/kaggle/working/mysitepackages/pylibjpeg_openjpeg/pylibjpeg_openjpeg-2.3.0-cp311-cp311-manylinux_2_17_x86_64.manylinux

In [3]:
import pydicom
from pydicom import dcmread
from pydicom.dataset import FileDataset, FileMetaDataset
from pydicom.uid import generate_uid, ImplicitVRLittleEndian

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
import polars as pl
import duckdb as dd
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
import pickle
import gc
import ctypes
from pathlib import Path
import logging
import json
import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor
import datetime
from typing import Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow import keras

In [4]:
df_all_data = pl.read_parquet('/kaggle/input/rsna-aneurysm-train-metadata-suman/full_training_data_v2.parquet')
print("Shape of training metadata", df_all_data.shape)
df_all_data.columns

Shape of training metadata (1000771, 28)


['aneurysm_position',
 'file_name',
 'file_path',
 'image_shape',
 'coordinates_x',
 'coordinates_y',
 'coordinates_f',
 'min_max_diff',
 'aneurysm_present_in_series',
 'aneurysm_present_in_image',
 'seriesinstanceuid',
 'patientage',
 'patientsex',
 'modality',
 'left_infraclinoid_internal_carotid_artery',
 'right_infraclinoid_internal_carotid_artery',
 'left_supraclinoid_internal_carotid_artery',
 'right_supraclinoid_internal_carotid_artery',
 'left_middle_cerebral_artery',
 'right_middle_cerebral_artery',
 'anterior_communicating_artery',
 'left_anterior_cerebral_artery',
 'right_anterior_cerebral_artery',
 'left_posterior_communicating_artery',
 'right_posterior_communicating_artery',
 'basilar_tip',
 'other_posterior_circulation',
 'aneurysm_position_encoded']

In [6]:
def process_mri_t2(image: np.ndarray):
    """
    Specific preprocessing for T2-weighted MRI
    - Bias field correction
    - Intensity normalization
    """
    # Convert to float32
    image = image.astype(np.float32)
    
    # Apply N4 bias field correction if needed
    # Note: This would require SimpleITK, commented out for simplicity
    # image = self._apply_bias_correction(image)
    
    # Z-score normalization
    mean = np.mean(image)
    std = np.std(image)
    image = (image - mean) / (std + 1e-10)
    
    # Scale to [0,1] range
    image = (image - np.min(image)) / (np.max(image) - np.min(image) + 1e-10)
    
    return image

def process_mri_t1_post(image: np.ndarray):
    """
    Specific preprocessing for T1 post-contrast MRI
    - Enhancement of contrast regions
    - Intensity standardization
    """
    image = image.astype(np.float32)
    
    # Enhance contrast
    p2, p98 = np.percentile(image, (2, 98))
    image = np.clip(image, p2, p98)
    
    # Normalize to [0,1]
    image = (image - p2) / (p98 - p2 + 1e-10)
    
    return image

def process_cta(image: np.ndarray, dcm: pydicom.dataset.FileDataset):
    """
    Specific preprocessing for CTA
    - Window/level optimization for vessel visualization
    - Vessel enhancement
    """
    # Get window settings (typical for CTA)
    window_center = 100  # Typical for CTA
    window_width = 700   # Typical for CTA
    
    # Override with DICOM values if available
    if hasattr(dcm, 'WindowCenter') and hasattr(dcm, 'WindowWidth'):
        window_center = dcm.WindowCenter
        window_width = dcm.WindowWidth
        
        # Handle multiple window settings
        if isinstance(window_center, pydicom.multival.MultiValue):
            window_center = window_center[0]
        if isinstance(window_width, pydicom.multival.MultiValue):
            window_width = window_width[0]
    
    # Apply window/level
    min_value = window_center - window_width // 2
    max_value = window_center + window_width // 2
    image = np.clip(image, min_value, max_value)
    
    # Normalize to [0,1]
    image = (image - min_value) / (max_value - min_value + 1e-10)
    
    return image

def process_mra(image: np.ndarray):
    """
    Specific preprocessing for MRA
    - Maximum intensity projection consideration
    - Vessel enhancement
    """
    image = image.astype(np.float32)
    
    # Normalize based on tissue characteristics
    p0, p100 = np.percentile(image, (0, 100))
    image = (image - p0) / (p100 - p0 + 1e-10)
    
    # Optional: Vessel enhancement using Frangi filter
    # Note: This would require additional dependencies
    # image = self._apply_vessel_enhancement(image)
    
    return image

In [15]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a list of integers."""
    # Ensure value is a list
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def preprocess_image(dicom_path, f_coord):
    # Read the DICOM file using pydicom
    dcm = dcmread(dicom_path)
    # Extract the pixel array and convert to bytes
    image = dcm.pixel_array
    modality = dcm.Modality

    if modality == 'CTA':
        image = process_cta(image, dcm_path)
        
    if modality == 'MRA':
        image = process_mra(image)
        
    if modality == 'MRI T2':
        image = process_mri_t2(image)

    if modality == 'MRI T1post':
        image = process_mri_t1_post(image)

    if image.ndim == 2:
        specific_slice_tensor = image
    else:
        specific_slice_tensor = tf.slice(
            image,
            begin=[f_coord, 0, 0],
            size=[1, image.shape[1], image.shape[2]]
        )
        # Squeeze to remove the extra dimension of size 1
        specific_slice_tensor = tf.squeeze(specific_slice_tensor)
    
    expanded_image = tf.expand_dims(specific_slice_tensor, -1)
    expanded_image = tf.image.grayscale_to_rgb(expanded_image)
    expanded_image = tf.image.resize(expanded_image, (128,128))
    sqzd_image = tf.squeeze(expanded_image)
    return sqzd_image

def create_tf_example(dicom_path, label, f_coord):
    """Converts a DICOM image to a tf.train.Example."""
    sqzd_image = preprocess_image(dicom_path, f_coord)
    
    if isinstance(sqzd_image, tf.Tensor):
        sqzd_image = sqzd_image.numpy()
    
    # Create a features dictionary
    feature = {
        'image': _bytes_feature(sqzd_image.tobytes()),
        'labels': _int64_feature(label)
        # You can add other DICOM metadata here if needed
        # e.g., 'patient_id': _bytes_feature(dcm.PatientID.encode())
    }
    
    return tf.train.Example(features=tf.train.Features(feature=feature))

def write_to_tfrecord(dicom_paths, labels, f_coords, output_path):
    """Writes a list of DICOM file paths to a TFRecord file."""
    with tf.io.TFRecordWriter(output_path) as writer:
        for dicom_path, label, f_coord in zip(dicom_paths, labels, f_coords):
            tf_example = create_tf_example(dicom_path, label, f_coord)
            writer.write(tf_example.SerializeToString())

In [ ]:
target_variables = ['left_infraclinoid_internal_carotid_artery',  'right_infraclinoid_internal_carotid_artery'
                    ,  'left_supraclinoid_internal_carotid_artery',  'right_supraclinoid_internal_carotid_artery'
                    ,  'left_middle_cerebral_artery',  'right_middle_cerebral_artery'
                    ,  'anterior_communicating_artery',  'left_anterior_cerebral_artery'
                    ,  'right_anterior_cerebral_artery',  'left_posterior_communicating_artery'
                    ,  'right_posterior_communicating_artery'
                    ,  'basilar_tip',  'other_posterior_circulation'
                    ,  'aneurysm_present_in_image'
                   ]

image_filenames = pl.Series(df_all_data.select(pl.col('file_path'))).to_list()
image_f_coords = pl.Series(df_all_data.select(pl.col('coordinates_f'))).fill_null(0).to_list()
image_labels = pl.Series(df_all_data.select(target_variables)).map_elements(lambda d: list(d.values())).to_list()

write_to_tfrecord(image_filenames, image_labels, image_f_coords, "training_data.tfrecord")